In [23]:
from suds.client import Client
from suds.transport.http import HttpTransport
import urllib2

In [24]:
class HTTPSudsPreprocessor(urllib2.BaseHandler):
    def __init__(self, SID):
        self.SID = SID

    def http_request(self, req):
        req.add_header('cookie', 'SID="'+self.SID+'"')
        return req

    https_request = http_request

In [25]:
class WokmwsSoapClient():
    """
    main steps you have to do:
        soap = WokmwsSoapClient()
        results = soap.search(...)
    """
    def __init__(self):
        self.url = self.client = {}
        self.SID = ''

        #self.url['auth'] = 'http://search.isiknowledge.com/esti/wokmws/ws/WOKMWSAuthenticate?wsdl'
        self.url['auth'] = 'http://search.webofknowledge.com/esti/wokmws/ws/WOKMWSAuthenticate?wsdl'
        #self.url['search'] = 'http://search.isiknowledge.com/esti/wokmws/ws/WokSearchLite?wsdl'
        self.url['search'] = 'http://search.webofknowledge.com/esti/wokmws/ws/WokSearchLite?wsdl'

        self.prepare()

    def __del__(self):
        self.close()

    def prepare(self):
        """does all the initialization we need for a request"""
        self.initAuthClient()
        self.authenticate()
        self.initSearchClient()

    def initAuthClient(self):
        self.client['auth'] = Client(self.url['auth'])

    def initSearchClient(self):
        http = HttpTransport()
        opener = urllib2.build_opener(HTTPSudsPreprocessor(self.SID))
        http.urlopener = opener
        self.client['search'] = Client(self.url['search'], transport = http)

    def authenticate(self):
        self.SID = self.client['auth'].service.authenticate()

    def close(self):
        self.client['auth'].service.closeSession()

    def search(self, query):
        qparams = {
            'databaseID' : 'WOS',
            'userQuery' : query,
            'queryLanguage' : 'en',
            'editions' : [{
                'collection' : 'WOS',
                'edition' : 'SCI',
            },{
                'collection' : 'WOS',
                'edition' : 'SSCI',
            }]
        }

        rparams = {
            'count' : 5, # 1-100
            'firstRecord' : 1,
            'fields' : [{
                'name' : 'Relevance',
                'sort' : 'D',
            }],
        }

        return self.client['search'].service.search(qparams, rparams)

In [26]:
soap = WokmwsSoapClient()

In [27]:
#from wos import *
#soap = WokmwsSoapClient()
results = soap.search('AU=Hallam')

TypeNotFound: Type not found: 'databaseID'

In [14]:
from suds.client import Client
url = 'http://search.webofknowledge.com/esti/wokmws/ws/WokSearch?wsdl'
client = Client(url)

In [15]:
client.service.retrieve('AU=Hallam', '')

ERROR:suds.client:<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:ns0="http://woksearch.v3.wokmws.thomsonreuters.com" xmlns:ns1="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/">
   <SOAP-ENV:Header/>
   <ns1:Body>
      <ns0:retrieve>
         <queryId>AU=Hallam</queryId>
         <retrieveParameters></retrieveParameters>
      </ns0:retrieve>
   </ns1:Body>
</SOAP-ENV:Envelope>


WebFault: Server raised fault: 'Unmarshalling Error: cvc-complex-type.2.4.b: The content of element 'retrieveParameters' is not complete. One of '{firstRecord}' is expected. '